In [28]:
import pandas as pd

from src.dataset import load_dataset

In [29]:
df_fact_check, df_posts, df_fact_check_mapping = load_dataset(data_folder="data/sample_data", trial=True)
df_fact_check = df_fact_check.reset_index()
df_posts = df_posts.reset_index()

df_posts.columns = ["ps_" + col if col not in ["post_id"] else col for col in df_posts.columns]
df_fact_check.columns = ["fc_" + col if col not in ["fact_check_id"] else col for col in df_fact_check.columns]

print(df_fact_check.shape, df_posts.shape, df_fact_check_mapping.shape)

(50, 4) (47, 5) (50, 3)


In [30]:
df_fact_check_mapping.head()

,post_id,fact_check_id,pair_lang
0,30,41,kor-eng
1,21,48,kor-eng
2,7,28,kor-eng
3,44,12,kor-eng
4,37,31,kor-eng


In [31]:
df_posts.head()

,post_id,ps_instances,ps_ocr,ps_verdicts,ps_text
0,0,"[(1593519481.0, ig)]",[],[],(La Filarmonica de Paris vivenciando un bolero...
1,1,"[(1620729739.0, ig)]",[(MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n...,[Partly False],
2,2,"[(1585630842.0, fb)]",[],[False information],(Así luce la Torre Latinoamericana de cdmx des...
3,3,"[(1648580403.0, fb)]",[],[False information],(El hombre fingió ser sordo y mudo durante 62 ...
4,4,"[(1515949200.0, fb)]",[],[Missing context],"(A PESAR DE SUS LOGROS, ESTO NO ES NOTICIA.\nH..."


In [32]:
df_fact_check.head()

,fact_check_id,fc_claim,fc_instances,fc_title
0,0,(South Africa's PRASA hiring workers for railw...,"[(1655739360.0, https://factcheck.afp.com/doc....",(Facebook posts falsely claim rail operator in...
1,1,(“Merck Scraps COVID Vaccines; Says It’s More ...,"[(1611619140.0, https://healthfeedback.org/cla...",(COVID-19 vaccines currently in use stimulate ...
2,2,"(Photos show NATO leaders in Madrid, Spain, Ph...","[(1657251780.0, https://factcheck.afp.com/doc....",(Doctored photos of world leaders with 'Satani...
3,3,"(Bus crash claims 18 lives in Milima, Zambia, ...","[(1579275420.0, https://factcheck.afp.com/no-d...",(No deaths were reported but several people su...
4,4,(U.S. pharmaceutical company Merck said it scr...,"[(1611871602.0, https://www.usatoday.com/story...",(Fact check: Merck discontinues COVID-19 vacci...


In [44]:
df_total = df_posts.merge(df_fact_check_mapping, on="post_id", how="inner")
df_total = df_total.merge(df_fact_check, on="fact_check_id", how="inner")
# df_total.rename(columns={"instances_x": "posts", "instances_y": "checker_link"}, inplace=True)

langs = df_total["pair_lang"].str.split("\-", expand=True)
df_total["ps_lang"] = langs[0]
df_total["fc_lang"] = langs[1]

df_total.drop(columns=["pair_lang"], inplace=True)

text_cols = ["fc_claim", "ps_text", "fc_title"]
for col in text_cols:
    df_total[f"{col}_source"] = df_total[col].apply(lambda x: x[0] if (type(x) in [list, tuple])&(len(x) > 0) else None)
    df_total[f"{col}_target"] = df_total[col].apply(lambda x: x[1] if (type(x) in [list, tuple])&(len(x) > 0) else None)
    df_total[f"{col}_langs"] = df_total[col].apply(lambda x: x[2] if (type(x) in [list, tuple])&(len(x) > 0) else None)

df_total.drop(columns=text_cols, inplace=True)

unitary_cols = ["ps_ocr", "ps_verdicts", "fc_instances"]
for col in unitary_cols:
    df_total[col] = df_total[col].apply(lambda x: x[0] if (type(x) in [list, tuple])&(len(x)>0) else None)

unitary_exp = ["ps_ocr"]
for col in unitary_exp:
    df_total[f"{col}_source"] = df_total[col].apply(lambda x: x[0] if (type(x) in [list, tuple]) else None)
    df_total[f"{col}_target"] = df_total[col].apply(lambda x: x[1] if (type(x) in [list, tuple]) else None)
    df_total[f"{col}_langs"] = df_total[col].apply(lambda x: x[2] if (type(x) in [list, tuple]) else None)

df_total.drop(columns=unitary_exp, inplace=True)

df_total.head(3)

,post_id,ps_instances,ps_verdicts,fact_check_id,fc_instances,ps_lang,fc_lang,fc_claim_source,fc_claim_target,fc_claim_langs,ps_text_source,ps_text_target,ps_text_langs,fc_title_source,fc_title_target,fc_title_langs,ps_ocr_source,ps_ocr_target,ps_ocr_langs
0,0,"[(1593519481.0, ig)]",None,18,"(1594243500.0, https://factual.afp.com/el-vide...",spa,spa,La filarmónica de París toca el Bolero de Rave...,The Paris Philharmonic plays Ravel's Bolero on...,"[(spa, 1.0)]",La Filarmonica de Paris vivenciando un bolero ...,The Philharmonic de Paris experiencing a boler...,"[(spa, 1.0)]",El video de una orquesta tocando el Bolero de ...,The video of an orchestra playing Ravel's Bole...,"[(spa, 1.0)]",None,None,None
1,1,"[(1620729739.0, ig)]",Partly False,26,"(1621021140.0, https://factcheck.afp.com/censu...",eng,eng,Census Bureau confirms conflict in total numbe...,Census Bureau confirms conflict in total numbe...,"[(eng, 1.0)]",None,None,None,"Census Bureau voter tally is self-reported, no...","Census Bureau voter tally is self-reported, no...","[(eng, 1.0)]",MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n.....,MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n.....,"[(eng, 0.9305148124694824), (fil, 0.0352872125..."
2,2,"[(1585630842.0, fb)]",False information,39,"(1585952220.0, https://factual.afp.com/este-ed...",spa,spa,La Torre Latinoamericana se ilumina al declara...,The Latin American Tower lights up when the he...,"[(spa, 1.0)]",Así luce la Torre Latinoamericana de cdmx desp...,This is what the CDMX Latin American Tower loo...,"[(spa, 1.0)]",Este edificio encendido no es la Torre Latinoa...,This lit building is not the Latin American To...,"[(spa, 1.0)]",None,None,None


In [42]:
df_total.ps_verdicts.value_counts()

ps_verdicts
False information           27
Partly false information     6
Missing context              5
Partly False                 1
Altered photo/video.         1
False information.           1
Altered photo                1
Missing context.             1
Name: count, dtype: int64

In [37]:
df_total["ps_ocr"]

0                                                    []
1     [(MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n...
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6                                                    []
7                                                    []
8     [("La esclavitud\nnunca tiene tanto\néxito com...
9                                                    []
10    [(이런한심한자가 대통령이란다.\n반공법폐지\n국정원폐지\n한미동맹반대\n통일연방제...
11                                                   []
12                                                   []
13    [(전라도에서만 국가유공자가\n50만이라니.이게\n말이되나?김대중.노무현 정권하에서...
14    [(새로운 조지아가이드스톤이 세워졌어요!!!\n유엔뉴스\n승인 2022.07.08 ...
15    [(새로운 조지아가이드스톤이 세워졌어요!!!\n유엔뉴스\n승인 2022.07.08 ...
16                                                   []
17                                              

In [20]:
print(df_total["ps_ocr"][~df_total["ps_ocr"].isnull()].to_markdown())

|    | ps_ocr                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|---:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [83]:
import numpy as np
df_total["claim"].map(list).tolist()

[['La filarmónica de París toca el Bolero de Ravel en una plataforma sobre el Sena por el levantamiento de la cuarentena',
  "The Paris Philharmonic plays Ravel's Bolero on a platform over the Seine for the lifting of the quarantine",
  [('spa', 1.0)]],
 ['Census Bureau confirms conflict in total number of voters in 2020 election',
  'Census Bureau confirms conflict in total number of voters in 2020 election',
  [('eng', 1.0)]],
 ['La Torre Latinoamericana se ilumina al declararse la emergencia sanitaria en México',
  'The Latin American Tower lights up when the health emergency is declared in Mexico',
  [('spa', 1.0)]],
 ['The man who faked being deaf and dumb for 62 years to avoid listening to his wife is now facing divorce from his wife after she found out the truth.',
  'The man who faked being deaf and dumb for 62 years to avoid listening to his wife is now facing divorce from his wife after she found out the truth.',
  [('eng', 1.0)]],
 ['Ha fallecido Gabriela Morreale y la prens

In [96]:
for i in range(3):
    print(df_total["checker_link"][i])

(1594243500.0, 'https://factual.afp.com/el-video-de-una-orquesta-tocando-el-bolero-de-ravel-sobre-el-sena-en-paris-es-de-2017#367fc73ea7c0c5812887632bc66ff2f5')
(1621021140.0, 'https://factcheck.afp.com/census-bureau-voter-tally-self-reported-not-official-election-data#ef1a31ac2b741f5621f5d1e962410144')
(1585952220.0, 'https://factual.afp.com/este-edificio-encendido-no-es-la-torre-latinoamericana-de-mexico-sino-el-empire-state-de-nueva-york#ccd19ef3f66db8c859c3c8005061ddff')


In [32]:
df_total["posts"][8]

[(1612380187.0, 'fb'), (1612450113.0, 'fb'), (1604783176.0, 'fb')]

In [36]:
df_total.map(lambda x: len(x) if type(x) in [list, tuple] else 1).describe()

,post_id,posts,ocr,verdicts,text,fact_check_id,claim,checker_link,title,source_lang,target_lang
count,50.0,50.000000,50.000000,50.00000,50.000000,50.0,50.0,50.0,50.0,50.0,50.0
mean,1.0,1.140000,0.260000,0.86000,2.680000,1.0,3.0,1.0,3.0,1.0,1.0
std,0.0,0.452205,0.443087,0.35051,0.740656,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.000000,0.000000,0.00000,1.000000,1.0,3.0,1.0,3.0,1.0,1.0
25%,1.0,1.000000,0.000000,1.00000,3.000000,1.0,3.0,1.0,3.0,1.0,1.0
50%,1.0,1.000000,0.000000,1.00000,3.000000,1.0,3.0,1.0,3.0,1.0,1.0
75%,1.0,1.000000,0.750000,1.00000,3.000000,1.0,3.0,1.0,3.0,1.0,1.0
max,1.0,3.000000,1.000000,1.00000,3.000000,1.0,3.0,1.0,3.0,1.0,1.0


In [45]:
df_total.to_csv("data/sample_data/proc/sample_data.csv", index=False)

In [46]:
pd.read_csv("data/sample_data/proc/sample_data.csv").head()

,post_id,ps_instances,ps_verdicts,fact_check_id,fc_instances,ps_lang,fc_lang,fc_claim_source,fc_claim_target,fc_claim_langs,ps_text_source,ps_text_target,ps_text_langs,fc_title_source,fc_title_target,fc_title_langs,ps_ocr_source,ps_ocr_target,ps_ocr_langs
0,0,"[(1593519481.0, 'ig')]",NaN,18,"(1594243500.0, 'https://factual.afp.com/el-vid...",spa,spa,La filarmónica de París toca el Bolero de Rave...,The Paris Philharmonic plays Ravel's Bolero on...,"[('spa', 1.0)]",La Filarmonica de Paris vivenciando un bolero ...,The Philharmonic de Paris experiencing a boler...,"[('spa', 1.0)]",El video de una orquesta tocando el Bolero de ...,The video of an orchestra playing Ravel's Bole...,"[('spa', 1.0)]",NaN,NaN,NaN
1,1,"[(1620729739.0, 'ig')]",Partly False,26,"(1621021140.0, 'https://factcheck.afp.com/cens...",eng,eng,Census Bureau confirms conflict in total numbe...,Census Bureau confirms conflict in total numbe...,"[('eng', 1.0)]",NaN,NaN,NaN,"Census Bureau voter tally is self-reported, no...","Census Bureau voter tally is self-reported, no...","[('eng', 1.0)]",MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n.....,MOM\nImahalo\nPatriot news\n@Grahmptr\n...\n.....,"[('eng', 0.9305148124694824), ('fil', 0.035287..."
2,2,"[(1585630842.0, 'fb')]",False information,39,"(1585952220.0, 'https://factual.afp.com/este-e...",spa,spa,La Torre Latinoamericana se ilumina al declara...,The Latin American Tower lights up when the he...,"[('spa', 1.0)]",Así luce la Torre Latinoamericana de cdmx desp...,This is what the CDMX Latin American Tower loo...,"[('spa', 1.0)]",Este edificio encendido no es la Torre Latinoa...,This lit building is not the Latin American To...,"[('spa', 1.0)]",NaN,NaN,NaN
3,3,"[(1648580403.0, 'fb')]",False information,37,"(1552003140.0, 'https://factly.in/no-man-did-n...",spa,eng,The man who faked being deaf and dumb for 62 y...,The man who faked being deaf and dumb for 62 y...,"[('eng', 1.0)]",El hombre fingió ser sordo y mudo durante 62 a...,The man pretended to be deaf and dumb for 62 y...,"[('spa', 1.0)]","No, man did not fake being deaf and dumb for 6...","No, man did not fake being deaf and dumb for 6...","[('eng', 1.0)]",NaN,NaN,NaN
4,4,"[(1515949200.0, 'fb')]",Missing context,30,"(1645529160.0, 'https://factual.afp.com/doc.af...",spa,spa,Ha fallecido Gabriela Morreale y la prensa ni ...,Gabriela Morreale has passed away and the pres...,"[('spa', 1.0)]","A PESAR DE SUS LOGROS, ESTO NO ES NOTICIA.\nHa...","DESPITE HIS ACHIEVEMENTS, THIS IS NOT NEWS.\nG...","[('spa', 1.0)]",La científica Morreale murió en 2017 y la pren...,The scientist Morreale died in 2017 and the pr...,"[('spa', 1.0)]",NaN,NaN,NaN
